In [1]:
import wandb
from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator
from fdd_defense.attacks import FGSMAttack
from fdd_defense.models.base import BaseTorchModel
from fdd_defense.models import LinearModel
import torch. nn as nn
from torch.optim import Adam
import torch
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from fdd_defense.defenders import DistillationDefender, QuantizationDefender, AdversarialTrainingDefender
dataset = FDDDataset(name='small_tep')
dataset.df.head()
scaler = StandardScaler()
scaler.fit(dataset.df[dataset.train_mask])
dataset.df[:] = scaler.transform(dataset.df)

Reading data/small_tep/test_mask.csv: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153300/153300 [00:00<00:00, 3746660.00it/s]


In [2]:
window_size = 10

fddmodel = LinearModel(
        window_size=window_size,
        step_size=1,
        num_epochs=29,
        batch_size=166,
        lr=0.004209,
    )
fddmodel.fit(dataset)

Creating sequence of samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 3506.11it/s]


Epochs ...:   0%|          | 0/29 [00:00<?, ?it/s]

Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 1, Loss: 1.2981


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

...

Epoch 27, Loss: 0.9643


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 28, Loss: 0.9633


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 29, Loss: 0.9624


In [3]:
test_loader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.test_mask,
    labels=dataset.labels,
    window_size=window_size,
    step_size=1,
)
for sample, index, label in test_loader:
    pred = fddmodel.predict(sample)
pred = pd.Series(pred, index=index, name='pred')

evaluator = FDDEvaluator(step_size=1)
evaluator.print_metrics(label, pred)

Creating sequence of samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 3290.04it/s]


FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9872/0.0000
    Fault 02: 0.9790/0.0001
    Fault 03: 0.0003/0.0000
    Fault 04: 0.9827/0.0000
    Fault 05: 0.9677/0.0000
    Fault 06: 0.8493/0.0000
    Fault 07: 0.9990/0.0000
    Fault 08: 0.4836/0.0001
    Fault 09: 0.0000/0.0000
    Fault 10: 0.0008/0.0000
    Fault 11: 0.0005/0.0000
    Fault 12: 0.2478/0.0000
    Fault 13: 0.3985/0.0002
    Fault 14: 0.0003/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0060/0.0000
    Fault 17: 0.9434/0.0000
    Fault 18: 0.8778/0.0000
    Fault 19: 0.0033/0.0000
    Fault 20: 0.7947/0.0000
Detection TPR: 0.5571
Detection FPR: 0.0003
Average Detection Delay (ADD): 116.74
Total Correct Diagnosis Rate (Total CDR): 0.8545

Clustering metrics
-----------------
Adjusted Rand Index (ARI): 0.2427
Normalized Mutual Information (NMI): 0.6315
Unsupervised Clustering Accuracy (ACC): 0.5803


In [6]:
defender = DistillationDefender(fddmodel)
attack = FGSMAttack(model=defender.fddmodel, eps=0.05)
for sample, index, label in test_loader:
    _sample = torch.FloatTensor(sample)
    _label = torch.LongTensor(label.values)
adv_sample = attack.attack(_sample, _label)

Training a student...


Epochs ...:   0%|          | 0/29 [00:00<?, ?it/s]

Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 1, Loss: 2.6568


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

...

Epoch 27, Loss: 1.0306


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 28, Loss: 1.0188


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 29, Loss: 1.0075


In [7]:
pred = defender.predict(adv_sample)
pred = pd.Series(pred, index=index, name='pred')

evaluator = FDDEvaluator(step_size=1)
evaluator.print_metrics(label, pred)

FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9827/0.0000
    Fault 02: 0.9687/0.0000
    Fault 03: 0.0000/0.0000
    Fault 04: 0.0340/0.0000
    Fault 05: 0.7334/0.0000
    Fault 06: 0.8273/0.0000
    Fault 07: 0.9947/0.0000
    Fault 08: 0.1189/0.0001
    Fault 09: 0.0000/0.0000
    Fault 10: 0.0000/0.0000
    Fault 11: 0.0000/0.0000
    Fault 12: 0.1499/0.0000
    Fault 13: 0.1026/0.0000
    Fault 14: 0.0003/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0000/0.0000
    Fault 17: 0.8008/0.0000
    Fault 18: 0.4671/0.0000
    Fault 19: 0.0000/0.0000
    Fault 20: 0.4150/0.0000
Detection TPR: 0.4144
Detection FPR: 0.0001
Average Detection Delay (ADD): 58.99
Total Correct Diagnosis Rate (Total CDR): 0.7958

Clustering metrics
-----------------
Adjusted Rand Index (ARI): 0.1374
Normalized Mutual Information (NMI): 0.5055
Unsupervised Clustering Accuracy (ACC): 0.4661


tpr 41%  fpr 0.01% cdr 79%